In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define the path to the dataset
dataset_path = '/content/drive/MyDrive/fruit_dataset_new'

# Import necessary libraries
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the data generators
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Build a simple CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator, steps_per_epoch=len(train_generator), validation_data=validation_generator, validation_steps=len(validation_generator), epochs=10)

# Save the model
model.save('/content/drive/MyDrive/fruit_classification_model.keras')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 2170 images belonging to 6 classes.
Found 540 images belonging to 6 classes.
Epoch 1/10
68/68 [==============================] - 23s 265ms/step - loss: 1.5751 - accuracy: 0.3866 - val_loss: 1.0689 - val_accuracy: 0.5778
Epoch 2/10
68/68 [==============================] - 19s 279ms/step - loss: 1.0165 - accuracy: 0.6111 - val_loss: 0.6835 - val_accuracy: 0.7222
Epoch 3/10
68/68 [==============================] - 18s 257ms/step - loss: 0.8131 - accuracy: 0.7060 - val_loss: 0.5750 - val_accuracy: 0.7963
Epoch 4/10
68/68 [==============================] - 21s 303ms/step - loss: 0.6605 - accuracy: 0.7581 - val_loss: 0.5156 - val_accuracy: 0.7907
Epoch 5/10
68/68 [==============================] - 17s 247ms/step - loss: 0.5759 - accuracy: 0.8037 - val_loss: 0.4365 - val_accuracy: 0.8796
Epoch 6/10
68/68 [==============================] - 17s 252ms/step - loss

In [ ]:
# Import necessary libraries
import numpy as np
from keras.preprocessing import image

# Load the trained model
loaded_model = keras.models.load_model('/content/drive/MyDrive/fruit_classification_model.keras')

# Load and preprocess the sample image
sample_image_path = '/content/drive/MyDrive/rotten_orange.png'
img = image.load_img(sample_image_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.

# Make predictions
predictions = loaded_model.predict(img_array)

# Get the predicted class
predicted_class = np.argmax(predictions)

# Define class labels
class_labels = {0: 'fresh_apple', 1: 'fresh_banana', 2: 'fresh_orange', 3: 'apple_rotten', 4: 'banana_rotten', 5: 'orange_rotten'}

# Display the results
print(f"Predicted Class: {class_labels[predicted_class]}")

1/1 [==============================] - 0s 78ms/step
Predicted Class: orange_rotten


In [ ]:
#LAB
import cv2
import os
import numpy as np

# Path to your dataset folder on Google Drive
dataset_path = '/content/drive/MyDrive/fruit_dataset_new'

# Initialize dictionaries to store LAB color values for each class
class_lab_values = {}

# Define a common size for LAB color arrays
common_size = (100, 100)  # Adjust to your desired size

# Function to extract LAB color features from an image and resize them
def extract_lab_color(image_path, target_size):
    image = cv2.imread(image_path)
    if image is None:
        return None
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    # Resize the LAB color array to the common size
    lab_image = cv2.resize(lab_image, target_size)
    return lab_image

# Loop through the images in the dataset folder
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(root, file)
            # Determine the class (e.g., 'rotten' or 'fresh') from the image path
            fruit_class = image_path.split("/")[-2]  # Adjust the index based on your folder structure
            if fruit_class not in class_lab_values:
                class_lab_values[fruit_class] = {'L': [], 'A': [], 'B': []}
            lab_color = extract_lab_color(image_path, common_size)
            if lab_color is not None:
                class_lab_values[fruit_class]['L'].extend(lab_color[:, :, 0].ravel())
                class_lab_values[fruit_class]['A'].extend(lab_color[:, :, 1].ravel())
                class_lab_values[fruit_class]['B'].extend(lab_color[:, :, 2].ravel())

# Print the range for the 50 most common values in each channel for each class
for fruit_class, values in class_lab_values.items():
    top_50_L = np.unique(np.array(values['L']))[:50]
    top_50_A = np.unique(np.array(values['A']))[:50]
    top_50_B = np.unique(np.array(values['B']))[:50]

    print(f"Class: {fruit_class}")
    print(f"L: {top_50_L.min()}-{top_50_L.max()}")
    print(f"A: {top_50_A.min()}-{top_50_A.max()}")
    print(f"B: {top_50_B.min()}-{top_50_B.max()}")
    print()


Class: freshapples
L: 0-49
A: 80-129
B: 113-162

Class: freshbanana
L: 0-49
A: 84-133
B: 100-149

Class: freshoranges
L: 0-49
A: 80-129
B: 89-138

Class: rottenapples
L: 0-49
A: 97-146
B: 103-152

Class: rottenbanana
L: 0-49
A: 82-132
B: 92-141

Class: rottenoranges
L: 0-49
A: 100-149
B: 85-134



In [ ]:
#LAB+regressor
import os
import cv2
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Path to your dataset folder
dataset_path = '/content/drive/MyDrive/fruit_dataset_new'

# Initialize a list to store LAB color features
lab_color_features = []
labels = []

# Function to extract LAB color features from an image
def extract_lab_color(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    mean_color = np.mean(lab_image, axis=(0, 1))
    return mean_color

# Loop through the images in the dataset folder
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(root, file)
            fruit_class = image_path.split("/")[-2]  # Adjust the index based on your folder structure
            color_features = extract_lab_color(image_path)
            if color_features is not None:
                lab_color_features.append(color_features)
                labels.append(fruit_class)

# Encode class labels into numerical values
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(lab_color_features, labels_encoded, test_size=0.2, random_state=42)

# Train a Random Forest Regressor to predict freshness scores
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = regressor.predict(X_test)

# Calculate the Mean Squared Error (MSE) as a measure of prediction accuracy
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 0.8724188191881919


In [ ]:
#LBP
import cv2
import os
import numpy as np
from collections import defaultdict
from skimage.feature import local_binary_pattern
# Path to your dataset folder on Google Drive
dataset_path = '/content/drive/MyDrive/fruit_dataset_new'

# Initialize dictionaries to store texture statistics for each class
class_textures = defaultdict(list)

# Function to extract LBP texture features from an image
def extract_lbp_texture(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        return None
    radius = 1  # Radius for LBP
    n_points = 8 * radius  # Number of points to sample
    lbp_image = local_binary_pattern(image, n_points, radius, method='uniform')
    # Calculate a histogram of the LBP image
    lbp_hist, _ = np.histogram(lbp_image.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    lbp_hist = lbp_hist.astype("float")
    lbp_hist /= (lbp_hist.sum() + 1e-7)  # Normalize the histogram
    return lbp_hist

# Loop through the images in the dataset folder
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(root, file)
            # Determine the class (e.g., 'rotten' or 'fresh') from the image path
            fruit_class = image_path.split("/")[-2]  # Adjust the index based on your folder structure
            lbp_hist = extract_lbp_texture(image_path)
            if lbp_hist is not None:
                class_textures[fruit_class].append(lbp_hist)

# Calculate and print the texture value ranges for each class
for fruit_class, textures in class_textures.items():
    if len(textures) > 0:
        min_textures = np.min(textures, axis=0)
        max_textures = np.max(textures, axis=0)
        print(f"Class: {fruit_class}")
        print("Minimum LBP Texture Values:", min_textures)
        print("Maximum LBP Texture Values:", max_textures)
        print()

Class: freshapples
Minimum LBP Texture Values: [0.00099668 0.0057302  0.00221339 0.04802971 0.08078672 0.10634569
 0.02810269 0.02284644 0.19209861 0.00836415]
Maximum LBP Texture Values: [0.01615093 0.03759158 0.03440238 0.12157012 0.33330485 0.30517857
 0.06871583 0.08051986 0.58624667 0.05476715]

Class: freshoranges
Minimum LBP Texture Values: [0.00181679 0.00584733 0.00327735 0.03408142 0.06388295 0.06462595
 0.01802545 0.02307793 0.06139933 0.01076845]
Maximum LBP Texture Values: [0.02353783 0.03851216 0.03389388 0.13364651 0.29723811 0.28683036
 0.08332199 0.07167025 0.77357761 0.07074568]

Class: rottenoranges
Minimum LBP Texture Values: [0.00531602 0.01245934 0.01171875 0.06067434 0.08766447 0.0790625
 0.03576231 0.02516454 0.04561873 0.02525144]
Maximum LBP Texture Values: [0.05946712 0.08193027 0.06109694 0.1523029  0.31575971 0.2853541
 0.09073265 0.08422619 0.51869398 0.14771825]

Class: rottenapples
Minimum LBP Texture Values: [0.00159593 0.00769918 0.00310284 0.04936207 

In [ ]:
#LBP+regressor
import cv2
import os
import numpy as np
from collections import defaultdict
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
# Path to your dataset folder on Google Drive
dataset_path = '/content/drive/MyDrive/fruit_dataset_new'

# Initialize dictionaries to store texture statistics for each class
class_textures = defaultdict(list)

# Function to extract LBP texture features from an image
def extract_lbp_texture(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        return None
    radius = 1  # Radius for LBP
    n_points = 8 * radius  # Number of points to sample
    lbp_image = local_binary_pattern(image, n_points, radius, method='uniform')
    # Calculate a histogram of the LBP image
    lbp_hist, _ = np.histogram(lbp_image.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    lbp_hist = lbp_hist.astype("float")
    lbp_hist /= (lbp_hist.sum() + 1e-7)  # Normalize the histogram
    return lbp_hist

# Loop through the images in the dataset folder
combined_features = []  # List to store LBP texture features
labels = []  # List to store class labels

for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(root, file)
            # Determine the class (e.g., 'rotten' or 'fresh') from the image path
            fruit_class = image_path.split("/")[-2]  # Adjust the index based on your folder structure
            lbp_hist = extract_lbp_texture(image_path)
            if lbp_hist is not None:
                combined_features.append(lbp_hist)
                labels.append(fruit_class)

# Encode class labels into numerical values
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, labels_encoded, test_size=0.2, random_state=42)

# Train a regression model (Random Forest Regressor) to predict freshness scores
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = regressor.predict(X_test)

# Calculate the Mean Squared Error (MSE) as a measure of prediction accuracy
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 1.2680151291512913


NameError: ignored

In [ ]:
#LAB+LBP+regressor_rf
import cv2
import os
import numpy as np
from collections import defaultdict
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import joblib

# Path to your dataset folder
dataset_path = '/content/drive/MyDrive/fruit_dataset_new'

# Initialize dictionaries to store color and texture statistics for each class
class_colors = defaultdict(list)
class_textures = defaultdict(list)

# Function to extract LAB color features from an image
def extract_lab_color(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    mean_color = np.mean(lab_image, axis=(0, 1))
    return mean_color

# Function to extract LBP texture features from an image
def extract_lbp_texture(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        return None
    radius = 1  # Radius for LBP
    n_points = 8 * radius  # Number of points to sample
    lbp_image = local_binary_pattern(image, n_points, radius, method='uniform')
    # Calculate a histogram of the LBP image
    lbp_hist, _ = np.histogram(lbp_image.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    lbp_hist = lbp_hist.astype("float")
    lbp_hist /= (lbp_hist.sum() + 1e-7)  # Normalize the histogram
    return lbp_hist

# Loop through the images in the dataset folder
combined_features = []  # List to store combined color and texture features
labels = []  # List to store class labels

for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(root, file)
            # Determine the class (e.g., 'rotten' or 'fresh') from the image path
            fruit_class = image_path.split("/")[-2]  # Adjust the index based on your folder structure
            lab_color = extract_lab_color(image_path)
            lbp_hist = extract_lbp_texture(image_path)
            if lab_color is not None and lbp_hist is not None:
                combined_feature = np.concatenate((lab_color, lbp_hist))
                combined_features.append(combined_feature)
                labels.append(fruit_class)

# Encode class labels into numerical values
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, labels_encoded, test_size=0.2, random_state=42)

# Train a regression model (Random Forest Regressor) to predict freshness scores
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train, y_train)

# Save the trained model to a file
model_filename = 'lab_lbp_regressor_model.joblib'
joblib.dump(regressor, model_filename)
print(f"Model saved as {model_filename}")

# Make predictions on the test set
y_pred = regressor.predict(X_test)

# Calculate the Mean Squared Error (MSE) as a measure of prediction accuracy
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Model saved as lab_lbp_regressor_model.joblib
Mean Squared Error: 0.49141217712177127


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#CNN_freshnesss
import os
import cv2
import numpy as np
from collections import defaultdict
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

# Path to your dataset folder on Google Drive
dataset_path = '/content/drive/MyDrive/fruit_dataset_new'

# Initialize dictionaries to store color and texture statistics for each class
class_colors = defaultdict(list)
class_textures = defaultdict(list)

# Function to extract LAB color features from an image
def extract_lab_color(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    mean_color = np.mean(lab_image, axis=(0, 1))
    return mean_color

# Function to extract LBP texture features from an image
def extract_lbp_texture(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        return None
    radius = 1
    n_points = 8 * radius
    lbp_image = local_binary_pattern(image, n_points, radius, method='uniform')
    lbp_hist, _ = np.histogram(lbp_image.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    lbp_hist = lbp_hist.astype("float")
    lbp_hist /= (lbp_hist.sum() + 1e-7)
    return lbp_hist

# Loop through the images in the dataset folder
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(root, file)
            fruit_class = image_path.split("/")[-2]  # Adjust the index based on your folder structure
            color_features = extract_lab_color(image_path)
            texture_features = extract_lbp_texture(image_path)
            if color_features is not None:
                class_colors[fruit_class].append(color_features)
            if texture_features is not None:
                class_textures[fruit_class].append(texture_features)

# Create combined feature vectors (color and texture) for each image
combined_features = []
labels = []

for fruit_class in class_colors.keys():
    for color_feature, texture_feature in zip(class_colors[fruit_class], class_textures[fruit_class]):
        combined_feature = np.concatenate((color_feature, texture_feature))
        combined_features.append(combined_feature)
        labels.append(fruit_class)

# Encode class labels into numerical values
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, labels_encoded, test_size=0.2, random_state=42)
X_train = np.array(X_train)
X_test = np.array(X_test)

print(X_train.shape)
print(X_test.shape)

# Define a CNN model for regression
n = len(combined_features[0])
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(n,)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the CNN model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the Mean Squared Error (MSE) as a measure of prediction accuracy
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# model.save('fruit_freshness_model.h5')
model.save('fruit_freshness_model.keras')



(2168, 13)
(542, 13)
Epoch 1/10
68/68 [==============================] - 1s 9ms/step - loss: 3.7840 - val_loss: 2.0671
Epoch 2/10
68/68 [==============================] - 0s 3ms/step - loss: 2.2186 - val_loss: 2.6182
Epoch 3/10
68/68 [==============================] - 0s 6ms/step - loss: 2.4846 - val_loss: 2.1806
Epoch 4/10
68/68 [==============================] - 0s 6ms/step - loss: 2.2212 - val_loss: 2.0501
Epoch 5/10
68/68 [==============================] - 0s 6ms/step - loss: 2.1641 - val_loss: 2.4617
Epoch 6/10
68/68 [==============================] - 0s 5ms/step - loss: 2.0686 - val_loss: 1.8973
Epoch 7/10
68/68 [==============================] - 0s 5ms/step - loss: 2.1413 - val_loss: 2.3031
Epoch 8/10
68/68 [==============================] - 0s 4ms/step - loss: 2.1265 - val_loss: 1.8690
Epoch 9/10
68/68 [==============================] - 0s 4ms/step - loss: 2.2657 - val_loss: 1.8771
Epoch 10/10
17/17 [==============================] - 0s 3ms/step
Mean Squared Error: 1.9914732480

In [ ]:
#CNN_freshness
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras

# Load the saved CNN model
model = keras.models.load_model('fruit_freshness_model.keras')

# Function to extract LAB color features from an image
def extract_lab_color(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    mean_color = np.mean(lab_image, axis=(0, 1))
    return mean_color

# Function to extract LBP texture features from an image
def extract_lbp_texture(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        return None
    radius = 1
    n_points = 8 * radius
    lbp_image = local_binary_pattern(image, n_points, radius, method='uniform')
    lbp_hist, _ = np.histogram(lbp_image.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    lbp_hist = lbp_hist.astype("float")
    lbp_hist /= (lbp_hist.sum() + 1e-7)
    return lbp_hist

# Function to predict freshness score
def predict_freshness(image_path):
    color_features = extract_lab_color(image_path)
    texture_features = extract_lbp_texture(image_path)

    if color_features is not None and texture_features is not None:
        combined_feature = np.concatenate((color_features, texture_features))
        combined_feature = np.array([combined_feature])

        # Predict the freshness score using the loaded CNN model
        freshness_score = model.predict(combined_feature)
        freshness_score_percentage = freshness_score[0][0] * 10
        return freshness_score_percentage
    else:
        return None

# Sample input image path
sample_image_path = '/content/drive/MyDrive/fresh_apple.png'

# Predict the freshness score
freshness_score = predict_freshness(sample_image_path)
if freshness_score is not None:
    print(f"Freshness Score: {freshness_score:.2f}%")
else:
    print("Unable to predict freshness for the provided image.")


1/1 [==============================] - 0s 54ms/step
Freshness Score: 28.67%


In [ ]:
#random_forest_freshness
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
import joblib

# Load the saved regression model
regressor = joblib.load('lbp_regressor_model.joblib')

# Function to extract LBP texture features from an image
def extract_lbp_texture(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        return None
    radius = 1
    n_points = 8 * radius
    lbp_image = local_binary_pattern(image, n_points, radius, method='uniform')
    lbp_hist, _ = np.histogram(lbp_image.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    lbp_hist = lbp_hist.astype("float")
    lbp_hist /= (lbp_hist.sum() + 1e-7)
    return lbp_hist

# Function to predict freshness score using the regression model
def predict_freshness(image_path):
    texture_features = extract_lbp_texture(image_path)

    if texture_features is not None:
        # Predict the freshness score using the loaded regression model
        freshness_score = regressor.predict([texture_features])
        return freshness_score[0]
    else:
        return None

# Sample input image path
sample_image_path = '/content/drive/MyDrive/rotten_apple.png'
# Predict the freshness score
freshness_score = predict_freshness(sample_image_path)
if freshness_score is not None:
    print(f"Freshness Score: {freshness_score:.2f}%")
else:
    print("Unable to predict freshness for the provided image.")


Freshness Score: 3.13%
